**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import skvideo
skvideo.setFFmpegPath('C:/Program Files (x86)/ffmpeg/ffmpeg-20190116-282a471-win64-static/bin')

import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import random

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Dropout
from keras.layers import Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e_i, e_f, n_games, epoch):
        e = e_i - epoch*(e_i-e_f)/n_games
        if e < e_f:
            e = e_f
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

There are two options for the agents: either 
- he exploits the best known act
- he explores a new one.  

Epsilon corresponds to the probability to explore a new action and not exploiting the best known action. It is essential for random exploration and allowing the agent to learn new strategies that may improve current policy.  
We can see that this option is enabled only while training: once we are testing, we only want to pick the best possible action at each time step.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
       # print('position')
       # print(self.position)
       # print('board')
       # print(self.board)
        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
       # print('state')
       # print(state)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
       # print('return')
       # print(state)
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The ```position``` array tracks the position of the rat.  
The ```board``` array stores the cheese and poisonous cell positions, and is updated if the rat moves on one of these cells (triggering the corresponding bonus/malus).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(4) # returns a random action between {0, 1, 2, 3}

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        game_over = False
        
        win = 0
        lose = 0
        
        while not game_over:
            
            action = agent.learned_act(state)
        
            state, reward, game_over = env.act(action)
            
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 6.5/13.0. Average score (-6.5)
Win/lose count 12.5/19.0. Average score (-6.5)
Win/lose count 12.5/10.0. Average score (-3.5)
Win/lose count 10.5/13.0. Average score (-3.25)
Win/lose count 12.5/13.0. Average score (-2.7)
Win/lose count 13.0/14.0. Average score (-2.4166666666666665)
Win/lose count 8.5/12.0. Average score (-2.5714285714285716)
Win/lose count 10.5/16.0. Average score (-2.9375)
Win/lose count 12.0/16.0. Average score (-3.0555555555555554)
Win/lose count 8.5/7.0. Average score (-2.6)
Final score: -2.6


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**A)**  
Let $\pi(.|s)$ denote the policy distribution, and $p(.|s,a)$ the transition probability distribution of states. The distribution of the couple $(s',a')$ given $(s,a)$ is given by :
\begin{equation*}
p(s',a'|s,a) = \pi(a'|s)p(s'|s,a)
\end{equation*}
$Q$ is defined as:

\begin{align*}
Q^\pi(s,a) &= E_{p^{\pi}}[\sum_{t = 0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
&= r(s,a) + \gamma E_{p^{\pi}}[\sum_{t = 0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] \\
&= r(s,a) + \gamma \sum_{(s',a')} \mathbb{P}(s_1 = s'|s_0 = s, a_0=a) \pi(a_0 = a'|s_0=s) E_{p^{\pi}}[\sum_{t = 0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a'] \\
&= r(s,a) + \gamma \sum_{(s',a')} p(s',a'|s,a) Q^{\pi}(s',a') \\
&= r(s,a) + \gamma E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')] \\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}  


Finally, $Q^{\pi}(s,a) = E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$  


**B)**  
We have, using **A)**:  
\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] =E_{s'\sim p(.|s,a)}[r(s,a)+\gamma E_{a'\sim \pi}[Q^{\pi}(s',a')]]
\end{equation*}
Hence:
\begin{equation*}
\begin{aligned}
Q^*(s,a) &= \max_{\pi}Q^{\pi}(s,a)\\
&= \max_{\pi}E_{s'\sim p(.|s,a)}[r(s,a)+\gamma E_{a'\sim \pi}[Q^{\pi}(s',a')]]\\
&= r(s,a)+\gamma \max_{\pi}E_{s'\sim p(.|s,a)}[E_{a'\sim \pi}[Q^{\pi}(s',a')]]\\
&=r(s,a)+\gamma E_{s'\sim \pi^*(.|s,a)}[E_{a'\sim \pi*}[Q^{*\pi}(s',a')]]\\
&= r(s,a)+\gamma E_{s'\sim \pi^*(.|s,a)}[Q^*(s',\pi^*(s'))] \\
&= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_a'Q^*(s',a')]
\end{aligned}
\end{equation*}
with $\pi^*(s) = argmax_a Q^*(s,a)$

$Q^*(s,a) = E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_a'Q^*(s',a')]$


**C)**  

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.insert(0, m)
        if len(self.memory) > self.max_memory: # if memory is saturated, we remove the oldest memory
            del self.memory[self.max_memory]

    def random_access(self):
        return random.choice(self.memory)

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1,epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape(1,s.shape[0],s.shape[1],s.shape[2]))) # returns best prediction

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        shape=(1,s_.shape[0],s_.shape[1],s_.shape[2])
        
        ## BUILDING BATCH
        for i in range(self.batch_size):
            transition=self.memory.random_access() # sampled memory with old_state, new_state and associated reward
            input_states[i]=transition[0] # old_state
            target_q[i]=self.model.predict(transition[0].reshape(shape)) # predicts rewards associated with each action
            if transition[4]: # game_over
                target_q[i,transition[2]]=transition[3] # the game finishes here, no need to discount future actions
            else:
                target_q[i,transition[2]]=transition[3]+self.discount*np.max(self.model.predict(transition[1].reshape(shape)))
                # add discounted future action
                
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model=Sequential()
        model.add(Flatten(input_shape=(5, 5, 2))) 
        model.add(Dense(64, activation='relu')) 
        model.add(Dense(16, activation='relu'))
        model.add(Dense(4, activation='sigmoid'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.3, epsilon = 0.1, memory_size=5000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 001/100 | Loss 0.0003 | Win/lose count 3.0/2.0 (1.0)
Epoch 002/100 | Loss 0.0040 | Win/lose count 5.5/3.0 (2.5)
Epoch 003/100 | Loss 0.0086 | Win/lose count 3.5/3.0 (0.5)
Epoch 004/100 | Loss 0.0139 | Win/lose count 1.5/2.0 (-0.5)
Epoch 005/100 | Loss 0.0035 | Win/lose count 4.5/4.0 (0.5)
Epoch 006/100 | Loss 0.0003 | Win/lose count 1.5/4.0 (-2.5)
Epoch 007/100 | Loss 0.0080 | Win/lose count 1.0/6.0 (-5.0)
Epoch 008/100 | Loss 0.0071 | Win/lose count 2.5/0 (2.5)
Epoch 009/100 | Loss 0.0028 | Win/lose count 3.0/6.0 (-3.0)
Epoch 010/100 | Loss 0.0177 | Win/lose count 3.0/6.0 (-3.0)
Epoch 011/100 | Loss 0.0184 | Win/lose count 3.5/3.0 (0.5)
Epoch 012/100 | Loss 0.0025 | Win/lose count 6.0/3.0 (3.0)
Epoch 013/100 | Loss 0.0149 | Win/lose count 4.5/5.0 (-0.5)
Epoch 014/100 | Loss 0.0019 | Win/lose count 2.5/3.0 (-0.5)
Epoch 015/100 | Loss 0.0070 | Win/lose count 3.0/4.0 (-1.0)
Epoch 016/100 | Loss 0.0048 | Win/lose count 1.5/0 (1.5)
Epoch 017/100 | Loss 0.0030 | Win/lose count 2.5/7.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, (2, 2), activation='relu', input_shape=(5,5,self.n_state,)))
        model.add(Conv2D(32, (2, 2), activation='relu'))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 001/100 | Loss 0.0063 | Win/lose count 4.5/7.0 (-2.5)
Epoch 002/100 | Loss 0.0097 | Win/lose count 1.5/0 (1.5)
Epoch 003/100 | Loss 0.0016 | Win/lose count 1.5/3.0 (-1.5)
Epoch 004/100 | Loss 0.0080 | Win/lose count 5.0/7.0 (-2.0)
Epoch 005/100 | Loss 0.0123 | Win/lose count 1.5/0 (1.5)
Epoch 006/100 | Loss 0.0100 | Win/lose count 5.5/6.0 (-0.5)
Epoch 007/100 | Loss 0.0059 | Win/lose count 3.0/5.0 (-2.0)
Epoch 008/100 | Loss 0.0067 | Win/lose count 4.5/5.0 (-0.5)
Epoch 009/100 | Loss 0.0068 | Win/lose count 3.5/4.0 (-0.5)
Epoch 010/100 | Loss 0.0108 | Win/lose count 4.0/1.0 (3.0)
Epoch 011/100 | Loss 0.0129 | Win/lose count 1.5/2.0 (-0.5)
Epoch 012/100 | Loss 0.0075 | Win/lose count 3.5/4.0 (-0.5)
Epoch 013/100 | Loss 0.0061 | Win/lose count 8.0/4.0 (4.0)
Epoch 014/100 | Loss 0.0082 | Win/lose count 3.0/8.0 (-5.0)
Epoch 015/100 | Loss 0.0038 | Win/lose count 2.5/5.0 (-2.5)
Epoch 016/100 | Loss 0.0046 | Win/lose count 6.0/2.0 (4.0)
Epoch 017/100 | Loss 0.0091 | Win/lose count 3.0/

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.0/0. Average score (2.0)
Win/lose count 2.5/0. Average score (2.25)
Win/lose count 4.5/1.0. Average score (2.6666666666666665)
Win/lose count 1.0/0. Average score (2.25)
Win/lose count 2.5/0. Average score (2.3)
Win/lose count 0/0. Average score (1.9166666666666667)
Win/lose count 4.0/0. Average score (2.2142857142857144)
Win/lose count 1.0/0. Average score (2.0625)
Win/lose count 0.5/0. Average score (1.8888888888888888)
Win/lose count 1.5/0. Average score (1.85)
Final score: 1.85
Test of the FC
Win/lose count 1.0/1.0. Average score (0.0)
Win/lose count 2.0/2.0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 1.0/2.0. Average score (-0.25)
Win/lose count 2.5/3.0. Average score (-0.3)
Win/lose count 1.5/2.0. Average score (-0.3333333333333333)
Win/lose count 2.5/3.0. Average score (-0.35714285714285715)
Win/lose count 1.5/2.0. Average score (-0.375)
Win/lose count 0.5/3.0. Average score (-0.6111111111111112)
Win/lose count 2.0

In [16]:
#HTML(display_videos('cnn_test10.mp4'))

In [17]:
#HTML(display_videos('fc_test10.mp4'))

**Positive** aspects:
- Both networks are better than the Random Agent
- Average score of both networks is positive at the end of the 10 games

**Comparison**:

| 10 games       | CNN | FC  | Random |
|----------------|-----|-----|--------|
| Average score  | 0.8 | 0.3 | -0.3   |
| Cheese Cells   | 38  | 18  | 51    |
| Poisoned Cells | 22   | 12  | 52.5   |
| Lost Games     | 22   | 2   | 4     |

It seems that CNN network is the one that performs the better


**Negative** aspects:  
- Scores are still far from the max possible score
- Both networks visit few cells

Both these issues can be solved by tackling the exploration/exploitation trade-off, in the next question.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
# We can also prefill the memory using the random agent: 
# https://becominghuman.ai/lets-build-an-atari-ai-part-1-dqn-df57e8ff3b26

def train_explore(agent, env, epoch, n_prefill, e_i, e_f, n_games, prefix=''):
    
    # Number of won games
    score = 0
    loss = 0
    
    for e in range(1,epoch+1):
        agent.set_epsilon(e_i, e_f, n_games, e) # We modify epsilon at each epoch
        
        # We prefill the memory with a random agent
        if e==1: # corresponds to the first game
            state = env.reset()
            game_over = False
            for _ in range(n_prefill):
                if game_over:
                    state = env.reset()
                action = np.random.randint(0, agent.n_action)
                prev_state = state
                state, reward, game_over = env.act(action, True)
                agent.memory.remember([prev_state, state, action, reward, game_over])
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0

        if train: # penalize only if training
            reward -= self.malus_position[self.x, self.y]

        reward += self.board[self.x, self.y]

        self.malus_position[self.x, self.y] = 0.1 # Penalization value for going on an already 
                                                  # visited cell set at 0.1
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                        self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1
        
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                        self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, n_prefill=2000, e_i=1, e_f=0.1, n_games=10, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 001/100 | Loss 0.0303 | Win/lose count 9.0/32.80000000000011 (-23.80000000000011)
Epoch 002/100 | Loss 0.0235 | Win/lose count 14.5/29.200000000000067 (-14.700000000000067)
Epoch 003/100 | Loss 0.0136 | Win/lose count 9.0/21.000000000000025 (-12.000000000000025)
Epoch 004/100 | Loss 0.0161 | Win/lose count 17.5/21.70000000000002 (-4.200000000000021)
Epoch 005/100 | Loss 0.0106 | Win/lose count 9.0/26.90000000000003 (-17.90000000000003)
Epoch 006/100 | Loss 0.0128 | Win/lose count 16.0/27.90000000000009 (-11.900000000000091)
Epoch 007/100 | Loss 0.0134 | Win/lose count 10.0/20.700000000000003 (-10.700000000000003)
Epoch 008/100 | Loss 0.0204 | Win/lose count 14.0/20.6 (-6.600000000000001)
Epoch 009/100 | Loss 0.0155 | Win/lose count 6.0/22.10000000000003 (-16.10000000000003)
Epoch 010/100 | Loss 0.0204 | Win/lose count 15.0/15.599999999999964 (-0.5999999999999641)
Epoch 011/100 | Loss 0.0208 | Win/lose count 22.0/14.499999999999968 (7.500000000000032)
Epoch 012/100 | Loss 0.0286 |

Epoch 094/100 | Loss 0.0129 | Win/lose count 19.5/15.79999999999997 (3.7000000000000295)
Epoch 095/100 | Loss 0.0226 | Win/lose count 21.0/15.599999999999984 (5.400000000000016)
Epoch 096/100 | Loss 0.0304 | Win/lose count 28.0/10.999999999999986 (17.000000000000014)
Epoch 097/100 | Loss 0.0257 | Win/lose count 27.5/9.699999999999985 (17.800000000000015)
Epoch 098/100 | Loss 0.0168 | Win/lose count 23.5/9.699999999999985 (13.800000000000015)
Epoch 099/100 | Loss 0.0209 | Win/lose count 22.5/19.700000000000017 (2.799999999999983)
Epoch 100/100 | Loss 0.0127 | Win/lose count 22.0/13.599999999999977 (8.400000000000023)


In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
#HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 23.0/0. Average score (23.0)
Win/lose count 22.5/3.0. Average score (21.25)
Win/lose count 16.5/2.0. Average score (19.0)
Win/lose count 21.0/1.0. Average score (19.25)
Win/lose count 23.0/3.0. Average score (19.4)
Win/lose count 25.5/2.0. Average score (20.083333333333332)
Win/lose count 17.0/1.0. Average score (19.5)
Win/lose count 21.5/1.0. Average score (19.625)
Win/lose count 12.0/1.0. Average score (18.666666666666668)
Win/lose count 17.5/2.0. Average score (18.35)
Final score: 18.35


With the 3 additional strategies explained above implemented, our agent is able to score much higher scores.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***